In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft

In [2]:
import os
import sys
# localization
currentdir = os.getcwd()

# Initial condition, sets

In [3]:
# inital sets, position of x0 and p0. One important remark we use now specifit walue of n0, m0. Stability island
#from previous classes.
M = 1000
n0 = 5
m0 = 400

x0 = 2*np.pi*n0/M
p0 = 2*np.pi*m0/M
heff = 2*np.pi/M # effective planck contants

n = np.arange(M) # corresponding to position (x)
m = np.arange(M) # corresponding to momentum (p)
# print(n)

# prepare the values of position (x) and momenta
x_list = np.zeros(M) # belongs to [0, 2*np.pi] each element
p_list = np.zeros(M) # belongs to [0, 2*np.pi] each element

for i in range(0, M):
    # position     
    x = 2*np.pi*n[i]/M
    x_list[i] = x
    # momenta
    p = 2*np.pi*m[i]/M
    p_list[i] = p

# Prepare normalized Gaussian wave packet: $\psi_{G} (x_{n})$

In [4]:
'''
We will prepare the normalised Gaussian wave packet. We will use following shortcuts:
    cal: calculate
    MoI: methof of images
    norm: normalisation
    f: factor
    
In this cell we just set some of functions, which we will use to find normalised Gaussian
wave packet
'''

################################ AMPLITUDE AND NORMALISATION ################################
def cal_amplitude(_wave):
    """
    Calculate the amplitude of taken single _wave function
    
    :param: _wave: single wave function [complex vector].
    return: the amplitude of _wave function [float]
    """
    amplitude = _wave.real * _wave.real + _wave.imag * _wave.imag
    return amplitude

def cal_norm(_sup_waves):
    """
    Find the normalisation of the superpositions of wave functions
    
    :param: _sup_waves: list of wave functions, which create superpositions wave function.
                        In different word we take into account every wave function, which
                        exists in system.
    return: normalisation factor
    """
    n_waves = len(_sup_waves) # number of waves
    f_norm = 0
    
    for i in range(0,n_waves):
        amplitude = cal_amplitude(_sup_waves[i])
        f_norm = f_norm + np.sqrt(amplitude)
    
    return f_norm

################################ SINGLE WAVE FUNCTION ################################
def psi0(x):
    """
    :param: x: descritised position
    return: single wave function [complex vector].
    """
    val_psi0 = np.exp(1j * p0 * x / heff) * np.exp(- (x - x0)**2 / (2 * heff))
    return val_psi0

def cal_wave_MoI(x):
    """
    take linear combination of some wave function: 'method of images'
    that gives us wave function, which is periodic.
    
    :param: x: descritised position
    return: superposition of few wave functions [complex vector]
    """
    # we will sum up for |d| = 4.
    val_psiG = 0
    for d in range(-4,4):
        val_psiG = val_psiG + psi0(x + 2*np.pi*d)
    
    return val_psiG

In [5]:
def normGaussianPacked(_x_list):
    """
    Find the normalised gaussian wave function
    
    :param: _x_list: list containg the values of position
    return: normalised gaussian wave factor list of [complex vector]
    """
    M = len(_x_list)
    psiG = np.zeros(M, dtype=complex) # complex vector
    
    #calculate gaussian wave package
    for i in range(0, M):
        x = _x_list[i]
        # calculate gaussian packaked
        val_psiG = cal_wave_MoI(x)
        psiG[i] = val_psiG
        
    # find normalised factor
    f_norm = cal_norm(psiG)
    
    # rescalled gaussian wave packed: Normalisation
    for i in range(0, M):
        psiG[i] = 1/f_norm * psiG[i]

    return psiG

# lest check
psiG = normGaussianPacked(x_list)
# print(psiG)

f_norm = cal_norm(psiG)
print('normalised factor of normalised gaussian wave function is (should be 1):', f_norm)

normalised factor of normalised gaussian wave function is (should be 1): 0.9999999999999996


# Do one step in gaussian packet evolution

In [6]:
"""
We calculate the quantum dynamics of our standard map. In another words
this time our gaussian wave packed will evolves.
"""
def pot_V(x, _K):
    """
    The phase vector, which depends on position. Phase vector for
    in the positional notation / space.
    
    :param: x: descritised position.
    :param: _K: some constant.
    return: Phase vector [complex vector]
    """
    val_V = np.exp(-1j/heff * _K * np.cos(x))
    return val_V

def Momenta_P(p):
    """
    The phase vector, which depends on momentum. Phase vector for
    in the momentum notation / space.
    
    :param: p: descritised momentum.
    return: Phase vector [complex vector]
    """
    val_P = np.exp(-1j/ (2*heff) * p**2)
    return val_P

def do_one_step(_wavef, _K):
    """
    Do one step in evolution our _wavef.
    
    :param: _wavef: normalised gaussian wave function
    :param: _K: some constant. Using for pot_V(x, _K).
    return: gaussian packed wave fanctions. List of [complex vector] after a 'one step'
            in evolution. One remark: after this evolution it will no longer be normalised!
    """
    psiG_evo = []
    # first step
    for i in range(0,M):
        x = x_list[i]
        # calculate
        _wavef[i] = _wavef[i] * pot_V(x, _K)

    # second step
    fft_psiG = fft(_wavef)

    # third step
    for i in range(0,M):
        p = p_list[i]
        # calculate
        fft_psiG[i] = fft_psiG[i] * Momenta_P(p)

    # fourth step
    psiG_evo = ifft(fft_psiG)
    return psiG_evo

In [7]:
# we prepare normalised gaussian w.f.
psiG = normGaussianPacked(x_list)
# do a one step
K = 0.0
psiG_evo = do_one_step(psiG, K)

# print(psiG_evo)
f_norm = cal_norm(psiG_evo)
print('normalised factor of normalised gaussian wave function is (should be 1):', f_norm)
print('after evolution our gaussian packed is stops being normalised.')

normalised factor of normalised gaussian wave function is (should be 1): 1.1892071150030448
after evolution our gaussian packed is stops being normalised.


# Show evolution of gaussian packed for K = 0.0

In [10]:
# inital sets. We will also x_list, y_list, n0, m0, etc (first cell)
step0 = 1
K = 0.0

################################ CREATE DIRECTORY TO PUT PLOTS ################################
output_plots_path = currentdir + '/K_' + str(K) + '_n0_' + str(n0) + '_m0_' + str(m0)
try:
    # Create target Directory
    os.mkdir(output_plots_path)
    print("Directory " , output_plots_path ,  " Created ")
except FileExistsError:
    print("Directory " , output_plots_path ,  " already exists")
        
################################ MAKE PLOTS  ################################
for picture in range(0,40):
    # for every picture you will make. Set how many you want to repeat 'steps in evolution'
    steps = step0 + picture
    # steps = step0 + picture * 2

    # initall
    ampl_evo = np.zeros(M)
    psiG_evo = normGaussianPacked(x_list)
    
    for i in range(0, steps):
        # new step in gaussian function
        psiG_evo = do_one_step(psiG_evo, K)

        for j in range(0,M):
            # for every wave in supperposition
            wave = psiG_evo[j]
            val_amp = cal_amplitude(wave)
            ampl_evo[j] = val_amp

    fig, ax = plt.subplots(figsize=(9.0, 6.0))
    ax.set_title(f'gaussian packed in step: {steps}', fontsize=22, loc='left')
    # data
    ax.plot(x_list, ampl_evo, label='probalibity of gaussian wave function')
    plt.axvline(x = x0, color = 'black', linestyle='dashed', label = f'x0 = {x0}')

    # describtion
    ax.set_ylabel(r"$|\psi(x_{n})|^2$", fontsize=18)
    ax.set_xlabel(r"$x_{n}$", fontsize=18)
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.grid(True)

    ax.legend()
    # plt.show()
    plt.savefig(output_plots_path + '/' + 'wave-move-' + str(picture) + '.png')

    plt.close(fig)

Directory  C:\Users\Krzysztof\Desktop\Magisterka\Semestr IV\Computer-modeling\Classes-2/K_0.0_n0_5_m0_400  already exists


# Show evolution of gaussian packed for K = 1.1

In [11]:
# inital sets. We will also x_list, y_list, n0, m0, etc (first cell)
step0 = 1
K = 1.1

################################ CREATE DIRECTORY TO PUT PLOTS ################################
output_plots_path = currentdir + '/K_' + str(K) + '_n0_' + str(n0) + '_m0_' + str(m0)
try:
    # Create target Directory
    os.mkdir(output_plots_path)
    print("Directory " , output_plots_path ,  " Created ")
except FileExistsError:
    print("Directory " , output_plots_path ,  " already exists")
        
################################ MAKE PLOTS  ################################
for picture in range(0,40):
    # for every picture you will make. Set how many you want to repeat 'steps in evolution'
    steps = step0 + picture
    # steps = step0 + picture * 2

    # initall
    ampl_evo = np.zeros(M)
    psiG_evo = normGaussianPacked(x_list)
    
    for i in range(0, steps):
        # new step in gaussian function
        psiG_evo = do_one_step(psiG_evo, K)

        for j in range(0,M):
            # for every wave in supperposition
            wave = psiG_evo[j]
            val_amp = cal_amplitude(wave)
            ampl_evo[j] = val_amp

    fig, ax = plt.subplots(figsize=(9.0, 6.0))
    ax.set_title(f'gaussian packed in step: {steps}', fontsize=22, loc='left')
    # data
    ax.plot(x_list, ampl_evo, label='probalibity of gaussian wave function')
    plt.axvline(x = x0, color = 'black', linestyle='dashed', label = f'x0 = {x0}')

    # describtion
    ax.set_ylabel(r"$|\psi(x_{n})|^2$", fontsize=18)
    ax.set_xlabel(r"$x_{n}$", fontsize=18)
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.grid(True)

    ax.legend()
    # plt.show()
    plt.savefig(output_plots_path + '/' + 'wave-move-' + str(picture) + '.png')

    plt.close(fig)

Directory  C:\Users\Krzysztof\Desktop\Magisterka\Semestr IV\Computer-modeling\Classes-2/K_1.1_n0_5_m0_400  Created 
